# Plate Velocity Basics

This notebook goes through a series of examples for how to use pygplates to work with plate velocities.

The basic requirements to calculate any plate velocities are:
  - a rotation file containing Euler poles defining plate motions
  - the plateid(s) that define the moving plate (and the frame of reference)
  - a time range over which to determine the velocity
  - the geographic location of the point where we evaluate velocity
  
### Example #1

The first example illustrates these concepts:
  - First, we load the rotation model from the Seton et al (2012) dataset
  - Second, we get the stage rotation that describes how the plate 101 (North America) is moving over the time period between 11 Ma and 10 Ma. [Note that since don't specify which plate North America is moving relative to, this calculation will default to getting the velcocity relative to the absolute reference frame in the rotation file]


In [11]:
import pygplates
import numpy as np

rotation_filename = 'Data/Seton_etal_ESR2012_2012.1.rot'

rotation_model = pygplates.RotationModel(rotation_filename)

timeFrom = 11.
timeTo = 10.
MovingPlate = 101

# Get the rotation from 11Ma to 10Ma, and the feature's reconstruction plate ID.
equivalent_stage_rotation = rotation_model.get_rotation(
    timeTo, MovingPlate, timeFrom)


- Having determined the stage rotation, define a specific point where we are going to evaluate the velocity 

In [12]:
velocity_point = pygplates.PointOnSphere((20,-60))

# Calculate a velocity for each reconstructed point over the 1My time interval.
velocity_vector = pygplates.calculate_velocities(
    velocity_point,
    equivalent_stage_rotation,
    1,
    pygplates.VelocityUnits.cms_per_yr)

velocity_magnitude_azimuth = pygplates.LocalCartesian.convert_from_geocentric_to_magnitude_azimuth_inclination(
                        velocity_point,
                        velocity_vector)

print(velocity_vector)
print(velocity_magnitude_azimuth)


[(1.9251545012319426, 5.157094761349058, 5.766929479762294e-17)]


Each entry in the list 'velocity_magnitude_azimuth' contains 3 values, where the first value is the magnitude, and the second is the azimuth:

In [13]:
print(('Velocity: magnitude = %0.4f cm/yr, azimuth = %0.4f' % \
            (velocity_magnitude_azimuth[0][0],np.degrees(velocity_magnitude_azimuth[0][1]))))

Velocity: magnitude = 1.9252 cm/yr, azimuth = 295.4798


### Example #2

The next cell is follows the same approach, but evaluates the velocity at a series of times. In this case, the point is assigned a plateid of 801 (Australia) and this point is then reconstructed relative to 802 (Antarctica), so that we can get the velocity of the same point on Australia

Note also that this second example also defines a 'delta' time, which controls the length of time that the cmputed velocity is averaged over (ie here the velocity is computed based on the motion for 0-10 Ma, 10-20 Ma, etc,  because the delta time is 10 Myr, you can choose this value). 

In [14]:
# Create for each point we want to reconstruct 
points = []
points.append((-30.,110.,801))

point_features = []
for lat, lon, plate_id in points:
    point_feature = pygplates.Feature()
    point_feature.set_geometry(pygplates.PointOnSphere(lat, lon))
    point_feature.set_reconstruction_plate_id(plate_id)
    point_features.append(point_feature)

max_time = 100.
delta_time = 10.

fixed_plate = 802

for time in np.arange(0,max_time+1.,delta_time):    
    # Reconstruct the point features.
    reconstructed_feature_geometries = []
    pygplates.reconstruct(point_features, rotation_model, reconstructed_feature_geometries, time)
    
    # Get the rotation from 'time+delta' to 'time', and the feature's reconstruction plate ID.
    equivalent_stage_rotation = rotation_model.get_rotation(
        time, plate_id, time+delta_time)
    
    for reconstructed_feature_geometry in reconstructed_feature_geometries:      
        # Calculate a velocity for each reconstructed point over the delta time interval.
        velocity_vector = pygplates.calculate_velocities(
            reconstructed_feature_geometry.get_reconstructed_geometry(),
            equivalent_stage_rotation,
            delta_time,
            pygplates.VelocityUnits.cms_per_yr)

        velocity_magnitude_azimuth = pygplates.LocalCartesian.convert_from_geocentric_to_magnitude_azimuth_inclination(
                                reconstructed_feature_geometry.get_reconstructed_geometry(),
                                velocity_vector)
    
        print(('Time = %0.2f Ma' % time))
        print(('Reconstructed Point Lat, Long = %s, %s' % \
            reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()))
        print(('Velocity: magnitude = %0.4f cm/yr, azimuth = %0.4f' % \
            (velocity_magnitude_azimuth[0][0],np.degrees(velocity_magnitude_azimuth[0][1]))))


Time = 0.00 Ma
Reconstructed Point Lat, Long = -29.999999999999996, 110.0
Velocity: magnitude = 6.2677 cm/yr, azimuth = 32.4431
Time = 10.00 Ma
Reconstructed Point Lat, Long = -34.7034363088108, 106.32036283887442
Velocity: magnitude = 6.4684 cm/yr, azimuth = 29.8713
Time = 20.00 Ma
Reconstructed Point Lat, Long = -39.7199742517607, 102.61917078417157
Velocity: magnitude = 7.3435 cm/yr, azimuth = 28.3382
Time = 30.00 Ma
Reconstructed Point Lat, Long = -45.44201210626845, 98.13386645231428
Velocity: magnitude = 6.3668 cm/yr, azimuth = 39.2918
Time = 40.00 Ma
Reconstructed Point Lat, Long = -49.785122680161045, 92.59187664759266
Velocity: magnitude = 0.8099 cm/yr, azimuth = 9.9017
Time = 50.00 Ma
Reconstructed Point Lat, Long = -50.50246213195913, 92.3950824324691
Velocity: magnitude = 5.5276 cm/yr, azimuth = 342.4757
Time = 60.00 Ma
Reconstructed Point Lat, Long = -55.26547781945862, 94.75324438392258
Velocity: magnitude = 0.8670 cm/yr, azimuth = 69.5545
Time = 70.00 Ma
Reconstructed Po